In [1]:
from enum import Enum
from typing import List
from getday import day

In [2]:
FileType = Enum('Type', [('FILE', 1), ('FREE', 2),])

def parse_input(example=True):
    return [int(c) for c in day(9, example)+'0']

def get_blocks(disk):
    '''Disk(get_blocks(parse_input(input)))
    '''
    blocks = []
    at = 0
    for i, (file_size, free_size) in enumerate(zip((disk)[::2], (disk)[1::2])):
        if file_size > 0:
            blocks.append(File(at, i, file_size))
            at += file_size
        if free_size > 0:
            blocks.append(Free(at, free_size))
            at += free_size
    return blocks
   

class Block:
    def __init__(self, at:int, file_type: FileType, size: int):
        self.at = at
        self.file_type = file_type
        self.size = size

class File(Block):
    def __init__(self, at:int, _id: int, size: int):
        super().__init__(at=at, file_type=FileType.FILE, size=size)
        self.id=_id
        
    def __repr__(self):
        return f'File(at={self.at}, id={self.id}, size={self.size})'

    def checksum(self):
        return sum([self.id * sum(range(self.at, self.at+self.size))])
        
class Free(Block):
    def __init__(self, at:int, size: int):
        super().__init__(at=at, file_type=FileType.FREE, size=size)
    def __repr__(self):
        return f'Free(at={self.at}, size={self.size})'

class Disk:
    def __init__(self, blocks: List[Block]):
        self.blocks = blocks

    def checksum(self):
        return sum([block.checksum() for block in self.blocks if isinstance(block, File)])

    def get_block_at(self, at):
        for block in self.blocks:
            if block.at == at:
                return block
        return False
        
    def disk_map(self):
        dm = ''
        at = 0
        while block := self.get_block_at(at):
            c = '.' if isinstance(block, Free) else str(block.id)
            dm += c * block.size
            at += block.size
            
        return dm

    def get_last_file(self):
        for i in range(len(self.blocks)-1, -1, -1):
            if isinstance(self.blocks[i], File):
                return self.blocks.pop(i)
        return False
        
    def find_free_space(self, at, size):
        for i, block in enumerate(self.blocks):
            if isinstance(block, Free) and block.size >= size and block.at < at:
                return block, i
        return False

    def defrag(self):
        dbg=False
        blocks = self.blocks
        
        file_buffer = self.get_last_file()
    
        new_blocks = []

        #for i in range(len(blocks)):
        i = -1
        while True:
            i += 1
            
            if i == len(self.blocks):
                break
            
            block = blocks[i]
            if dbg: print('=== Processing block', block)
            
            if isinstance(block, File):
                new_blocks.append(block)
                if dbg: print('  Added to new blocks:', new_blocks[-1])
                continue
    
            assert isinstance(block, Free)

            while block.size > 0 and block.at < file_buffer.at:
                if dbg: print('    --- Processing', block)
                if dbg: print('    --- file buffer', file_buffer)
                if block.size >= file_buffer.size:
                    if dbg: print(f'    File id {file_buffer.id}, size {file_buffer.size} fits in free space size {block.size}')
                    file_buffer.at = block.at
                    new_blocks.append(file_buffer)
                    if dbg: print(f'    Added to new blocks: file', new_blocks[-1])
                    block.size -= file_buffer.size
                    block.at += file_buffer.size
                    file_buffer = self.get_last_file()
                    if dbg: print(f'    Pop a new file buffer id {file_buffer}')
                else:  # free block size < file size
                    if dbg: print(f'    Free space {block.size} smaller than file {file_buffer.size}')
                    file_frag = File(at=block.at, _id=file_buffer.id, size=block.size)
                    if dbg: print('    Created a file frag:', file_frag)
                    file_buffer.size -= block.size
                    new_blocks.append(file_frag)
                    if dbg: print(f'    Added to new blocks:', new_blocks[-1])
                    if dbg: print(f'    Remaining file_buffer:', file_buffer)
                    block.size = 0

        # add last file buffer
        if file_buffer.size > 0:
            new_blocks.append(file_buffer)

        self.blocks = new_blocks
        
        
    def defrag2(self):
        dbg=False
        blocks = self.blocks
        new_blocks=[]
        while file := self.get_last_file():
            if dbg: print(f'{file=}')
            if block_i := self.find_free_space(file.at, file.size):
                block, i = block_i
                if dbg: print(f'Found enough free space at {block=}')
                self.blocks.append(Free(file.at, file.size))
                file.at = block.at
                block.size -= file.size
                block.at += file.size
                if block.size == 0:
                    if dbg: print(f'Deleting zero-sized free block', block)
                    del self.blocks[i]
                
                if dbg: print(f'File moved, new file {file=}')
            else:
                if dbg: print('No free space found.')
            new_blocks.append(file)
        
        new_blocks.extend([block for block in self.blocks if isinstance(block, Free)])        
        self.blocks = new_blocks
       

In [3]:
def part1(example=True):
    blocks = get_blocks(parse_input(example))
    disk = Disk(blocks)
    disk.defrag()
    if example: print(disk.disk_map(), '\n0099811188827773336446555566')
    return disk.checksum()

def part2(example=True):
    blocks = get_blocks(parse_input(example))
    disk = Disk(blocks)
    #print(disk.disk_map())
    disk.defrag2()
    if example: print(disk.disk_map(), '\n00992111777.44.333....5555.6666.....8888..')
    #print(disk.blocks)
    return disk.checksum()  

In [4]:
part1()
#1928

0099811188827773336446555566 
0099811188827773336446555566


1928

In [5]:
part2()
#2858

00992111777.44.333....5555.6666.....8888.. 
00992111777.44.333....5555.6666.....8888..


2858

In [6]:
%time part1(example=False)
#6337367222422

CPU times: user 1.12 s, sys: 0 ns, total: 1.12 s
Wall time: 1.12 s


6337367222422

In [7]:
%time part2(example=False)
#6361380647183

CPU times: user 17.1 s, sys: 17.9 ms, total: 17.1 s
Wall time: 17.2 s


6361380647183